# A short introduction on solving equation systems numerically for poromechanics

*[Written by: Maxime Pierre, 2023]* \
\
This series of notebooks aims at giving theoretical background on basic methods for solving linear and especially nonlinear systems of equations, as well as examples of their implementation in `python` with examples in poromechanical constitutive modelling. \
When implementing constitutive models, we are often faced with systems of equations that need to be solved, as many of the problems we are dealing with will be **inverse problems**. An inverse problem amounts to determining the causes to an observed outcome. \
Given a behaviour $f:x \rightarrow f(x)$ and an observed outcome $y$, we deal with the following problem:
$$
\text{Find} \ x \ \lvert \ f(x)=y.
$$
Let us take a very simple first example: consider a 1D bar element in linear elasticity. Its behaviour, linking the stress $\sigma$ with the deformation $\varepsilon$ is:
$$
\sigma = E(\varepsilon-\varepsilon_0) = f(\varepsilon),
$$
with $E$ the Young modulus of the material. Then, if we want to calculate the deformation corresponding to a given stress state $\tilde{\sigma}$, we must solve the inverse problem:
$$
\text{Find} \ \varepsilon \ \lvert \ E(\varepsilon-\varepsilon_0) = \tilde{\sigma}.
$$
Of course, this is a trivial problem since the relationship is linear, and is then easily inversed:
$$
\varepsilon = \varepsilon_0 + \frac{\tilde{\sigma}}{E}.
$$
However, the task may not always be this simple. What if we considered nonlinear elasticity? In the case of simple laws such as power laws, we could still manage it easily, but what about more complex functions, for which the inverse is unknown? We need other numerical tools that we will introduce starting from the next notebook. \
More often than not, we will also be dealing with multiple equations to be solved simultaneously: poromechanical problems are coupled. In order to illustrate this, let us take a simple isotropic porous solid with the usual state equations:
$$
\left\{
\begin{array}{}
\sigma = K\varepsilon -bp, \\
\phi - \phi_0 = b\varepsilon + \frac{p}{N}.
\end{array}
\right.
$$
If we consider a force-controlled, undrained isotropic compression with an incompressible fluid, we have for example:
$$
\left\{
\begin{array}{}
\sigma =-\tilde{\sigma}, \\
\phi = \phi_0.
\end{array}
\right.
$$
Finding the corresponding deformation and pressure requires solving the system:
$$
\left\{
\begin{array}{}
K\varepsilon -bp =-\tilde{\sigma}, \\
b\varepsilon + \frac{p}{N} = 0.
\end{array}
\right.
$$
We have two equations and two unknowns, which means we will find a unique solution unless our system is linked. Solving it seems easy enough: we can just express one unknown as a function of the other from the second equation, then inject it in the first, and voilà! That is indeed a decent way to approach that problem, and in fact close to the *Gauss-Seidel* algorithm which we will talk about in a minute. A useful representation of this problem consists in using matrices and vectors:
$$
\begin{bmatrix}
K & -b \\
b & \frac{1}{N}
\end{bmatrix}
\begin{bmatrix}
\varepsilon \\
p
\end{bmatrix}=
\begin{bmatrix}
-\tilde{\sigma} \\
0
\end{bmatrix},
$$
which is a problem of the usual form:
$$
K\cdot U = F
$$
of unknown $U$. We know an easy way of solving this problem: if we can calculate the inverse of $K$, we have:
$$
U = K^{-1}\cdot F.
$$
Let us take a simple example: $\tilde{\sigma}=1$ MPa, $K=1$ GPa, $b = 0.5$ and $N = 1$ GPa.

In [ ]:
import numpy as np
# Solicitation
sig_tilde = 1e6

# Material parameters
K = 1e9
b = 0.5
N = 1e9

### Exercise ###
# Write here the matrix K and vector F as numpy arrays. They should be of shape (2,2) and (2,), respectively.
# You can check the shape of an array A with A.shape

K = # Matrix K here
F = # Vector F here

Now that everything is defined, let us calculate our unknowns:

In [ ]:
### Exercise ###
# Get the solution U. You can use the following functions: np.linalg.inv and np.dot
# U should be of shape (2,)

# Calculating the inverse of K
K_inv = # Inverse K here
# Getting U
U = # Calculate U here

print("The corresponding state is : epsilon = {:.4f}, p = {} Pa.".format(*U))

In [ ]:
# Checking that KU=F
print("Result of K*U: sigma = {} Pa, phi - phi0 = {}.".format(*np.dot(K,U)))

## Bonus: a bit of matrix manipulation: Gaussian elimination algorithm

As an exercise to familiarize ourself with basic matrix manipulations in `python`, we can try and implement a simple way of solving linear systems of equations, however large they are, without using a built-in matrix inversion function. \
Gauss elimination consists of 3 basic operations:
- Swap two rows of the matrix,
- Multiply a row by a non-zero scalar,
- Add a multiple of one row to another row.

Say we are trying to solve the following linear system:
$$
\left\{\begin{array}{cl}
2x+y-z &=8, \\
4x+2y+2z &=-11, \\
-2x+y+2z &= -3.
\end{array}\right.
$$
We can build the corresponding *augmented matrix*:
$$
\left[
\begin{array}{ccc|c}
2 & 1 & -1 & 8 \\
4 & 2 & 2 & -11 \\
-2 & 1 & 2 & -3 \\
\end{array}
\right]
$$
The goal is to reduce the left $3\times 3$ part to identity to get the solution in the right part. We will first try to have a $1$ and two $0$ in the first column. \
We will start by making a $1$ in position $(1,1)$: we thus multiply the first row by $1/2$ (second operation):
$$
\left[
\begin{array}{ccc|c}
\color{red}{1} & \frac{1}{2} & -\frac{1}{2} & 4 \\
4 & 2 & 2 & -11 \\
-2 & 1 & 2 & -3 \\
\end{array}
\right]
$$
Then, we eliminate the first column in rows 2 and 3 by adding 3 times and 2 times the first row, respectively:
$$
\left[
\begin{array}{ccc|c}
\color{red}{1} & \frac{1}{2} & -\frac{1}{2} & 4 \\
\color{red}{0} & 0 & 4 & -27 \\
\color{red}{0} & 2 & 1 & 5 \\
\end{array}
\right]
$$
The next step is to get a $1$ in position $(2,2)$. However, we currently have $0$ there, so there is no way we can multiply row $2$ to get $1$ in that position. \
We will then swap it with the first row which has a non-zero value in the second column (here, row $3$):
$$
\left[
\begin{array}{ccc|c}
\color{red}{1} & \frac{1}{2} & -\frac{1}{2} & 4 \\
\color{red}{0} & 2 & 1 & 5 \\
\color{red}{0} & 0 & 4 & -27 \\
\end{array}
\right]
$$
Finally, we multiply the second row by $1/2$, eliminate the column $2$ in the row above, then with row $3$ again multiply by $1/4$ and eliminate other values in column $3$. We end up with:
$$
\left[
\begin{array}{ccc|c}
1 & 0 & 0 & -\frac{37}{16} \\
0 & 1 & 0 & \frac{47}{8} \\
0 & 0 & 1 & -\frac{27}{4} \\
\end{array}
\right]
$$
Let us implement the algorithm for any linear system:

In [ ]:
def gauss_elimination(A):
    # Here M is an "augmented" matrix, what matters is the number of rows
    M = A.copy() # Creating a copy of the original matrix, otherwise it will be modified by the function
    nb_rows = M.shape[0]
    for i in range(nb_rows):
        # Check if our pivot is 0
        if ### Exercise ###: # We need to swap rows
            j = i
            while ### Exercise: while rows have 0 in column i, we continue looking for subsequent row###:
                if j>=nb_rows:
                    return None
                else:
                    j+=1
            # Swap rows i and j
        
            ### Exercise: swap the rows ###
            ### Hint: you must use an auxiliary variable to store one of the rows while you are doing the swap ###
        
            print("Swapped rows {} and {}:".format(i,j))
            print(M)
        #Now our pivot is non-zero
        
        ### Exercise: Make our pivot value 1 (operation 2) ###
        
        print("Reduced pivot in row {} to 1:".format(i))
        print(M)
        for k in range(nb_rows):
            
            ### Exercise: for all rows other than i, use operation 3 to make value (k,i) zero ###
            
        print("Eliminated other values in column {}:".format(i))
        print(M)
    return M
            
# Defining our matrices
K = np.array([[2,1,-1],[4,2,2],[-2,1,2]], dtype=float)
F = np.array([8, -11, 3], dtype=float).reshape(3,1)
A = np.hstack((K,F)) # Augmented matrix by horizontally stacking the two others (hstack)

# Checking the algorithm against matrix inversion
M = gauss_elimination(A)
print("Solution from Gauss elimination: X={}".format(M[:,-1]))
sol = np.dot(np.linalg.inv(K), F).reshape(3,)
print("Actual solution: X={}".format(sol))
print("You succeeded!" if np.all(M[:,-1]==sol) else "Check your algorithm!")

Gauss elimination can also be used to invert a matrix: just use the previous, but instead of putting the $F$ vector on the right, put the identity matrix. Then, the right matrix will become the inverse of the initially left matrix:
$$
\left[\begin{array}{c|c}
M & I\\
\end{array}\right]
\rightarrow
\left[\begin{array}{c|c}
I & M^{-1}\\
\end{array}\right]
$$
Can you inverse the following matrix $M$?

In [ ]:
M = np.array([[4, 0, 2, 2, 3],
              [9, 3, 2, 0, 2],
              [0, 9, 1, 9, 7],
              [1, 1, 9, 4, 8],
              [7, 8, 9, 3, 3]], dtype=float)

# Make Identity Matrix
I = ### Exercise: make the I_5 matrix ###

M_inv = ### Exerise: calculate M^-1 with gauss elimination ###
print("Checking M^-1.M: {}".format(np.dot(M_inv,M)))